In [ ]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from math import log, exp
import re
from typing import List, Dict, Tuple
import random

# Download required NLTK data
nltk.download('brown')
nltk.download('punkt')
nltk.download('punkt_tab')  # Required for newer NLTK versions
nltk.download('stopwords')

from nltk.corpus import brown
from nltk.tokenize import word_tokenize, sent_tokenize

print("Setup completed successfully!")

class TextPreprocessor:
    """Class to handle text preprocessing for n-gram models"""

    def __init__(self):
        self.start_token = "<s>"
        self.end_token = "</s>"
        self.unk_token = "<unk>"

    def clean_text(self, text):
        """Clean and normalize text"""
        # Convert to lowercase
        text = text.lower()
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        # Keep only letters, numbers, and basic punctuation
        text = re.sub(r'[^\w\s.,!?;:]', '', text)
        return text.strip()

    def tokenize_sentences(self, text):
        """Tokenize text into sentences and words"""
        sentences = sent_tokenize(text)
        tokenized_sentences = []

        for sentence in sentences:
            sentence = self.clean_text(sentence)
            words = word_tokenize(sentence)
            if words:  # Only add non-empty sentences
                tokenized_sentences.append(words)

        return tokenized_sentences

    def add_sentence_boundaries(self, sentences, n):
        """Add sentence boundary tokens"""
        processed_sentences = []
        for sentence in sentences:
            # Add start tokens
            padded_sentence = [self.start_token] * (n-1) + sentence + [self.end_token]
            processed_sentences.append(padded_sentence)
        return processed_sentences

    def handle_unknown_words(self, sentences, vocab_threshold=2):
        """Replace rare words with UNK token"""
        # Count word frequencies
        word_counts = Counter()
        for sentence in sentences:
            for word in sentence:
                word_counts[word] += 1

        # Replace rare words with UNK
        processed_sentences = []
        for sentence in sentences:
            processed_sentence = []
            for word in sentence:
                if word_counts[word] >= vocab_threshold or word in [self.start_token, self.end_token]:
                    processed_sentence.append(word)
                else:
                    processed_sentence.append(self.unk_token)
            processed_sentences.append(processed_sentence)

        return processed_sentences

# Load and preprocess Brown Corpus
print("Loading Brown Corpus...")
brown_sents = brown.sents()
print(f"Total sentences in Brown Corpus: {len(brown_sents)}")

# Convert to string format and preprocess
preprocessor = TextPreprocessor()
corpus_text = ""
for sent in brown_sents[:5000]:  # Use first 5000 sentences for faster processing
    corpus_text += " ".join(sent) + " "

print("Preprocessing corpus...")
tokenized_sentences = preprocessor.tokenize_sentences(corpus_text)
print(f"Number of processed sentences: {len(tokenized_sentences)}")

# Split into train and test sets
train_size = int(0.8 * len(tokenized_sentences))
train_sentences = tokenized_sentences[:train_size]
test_sentences = tokenized_sentences[train_size:]

print(f"Training sentences: {len(train_sentences)}")
print(f"Test sentences: {len(test_sentences)}")


class NGramModel:
    """N-gram language model with various smoothing techniques"""

    def __init__(self, n, smoothing='laplace', alpha=1.0, discount=0.75):
        self.n = n
        self.smoothing = smoothing
        self.alpha = alpha  # Laplace smoothing parameter
        self.discount = discount  # Kneser-Ney discount parameter
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)
        self.vocabulary = set()
        self.vocab_size = 0

    def train(self, sentences):
        """Train the n-gram model"""
        print(f"Training {self.n}-gram model with {self.smoothing} smoothing...")

        # Add sentence boundaries
        preprocessor = TextPreprocessor()
        processed_sentences = preprocessor.add_sentence_boundaries(sentences, self.n)
        processed_sentences = preprocessor.handle_unknown_words(processed_sentences)

        # Count n-grams and contexts
        for sentence in processed_sentences:
            for word in sentence:
                self.vocabulary.add(word)

            for i in range(len(sentence) - self.n + 1):
                ngram = tuple(sentence[i:i + self.n])
                context = ngram[:-1]

                self.ngram_counts[ngram] += 1
                if self.n > 1:
                    self.context_counts[context] += 1

        self.vocab_size = len(self.vocabulary)
        print(f"Vocabulary size: {self.vocab_size}")
        print(f"Unique {self.n}-grams: {len(self.ngram_counts)}")

        # Special processing for Kneser-Ney smoothing
        if self.smoothing == 'kneser_ney' and self.n > 1:
            self._compute_kneser_ney_counts()

    def _compute_kneser_ney_counts(self):
        """Compute continuation counts for Kneser-Ney smoothing"""
        self.continuation_counts = defaultdict(int)
        self.continuation_contexts = defaultdict(set)

        for ngram in self.ngram_counts:
            if len(ngram) > 1:
                suffix = ngram[1:]
                context = ngram[:-1]
                self.continuation_contexts[suffix].add(context)

        for suffix, contexts in self.continuation_contexts.items():
            self.continuation_counts[suffix] = len(contexts)

    def get_probability(self, ngram):
        """Calculate probability of an n-gram"""
        if isinstance(ngram, str):
            ngram = tuple(ngram.split())

        if len(ngram) != self.n:
            raise ValueError(f"Expected {self.n}-gram, got {len(ngram)}-gram")

        if self.smoothing == 'laplace':
            return self._laplace_probability(ngram)
        elif self.smoothing == 'kneser_ney':
            return self._kneser_ney_probability(ngram)
        elif self.smoothing == 'good_turing':
            return self._good_turing_probability(ngram)
        else:
            return self._mle_probability(ngram)

    def _mle_probability(self, ngram):
        """Maximum Likelihood Estimation (no smoothing)"""
        if self.n == 1:
            total_count = sum(self.ngram_counts.values())
            return self.ngram_counts[ngram] / total_count if total_count > 0 else 0
        else:
            context = ngram[:-1]
            context_count = self.context_counts[context]
            return self.ngram_counts[ngram] / context_count if context_count > 0 else 0

    def _laplace_probability(self, ngram):
        """Laplace (Add-alpha) smoothing"""
        if self.n == 1:
            total_count = sum(self.ngram_counts.values())
            return (self.ngram_counts[ngram] + self.alpha) / (total_count + self.alpha * self.vocab_size)
        else:
            context = ngram[:-1]
            context_count = self.context_counts[context]
            return (self.ngram_counts[ngram] + self.alpha) / (context_count + self.alpha * self.vocab_size)

    def _kneser_ney_probability(self, ngram):
        """Simplified Kneser-Ney smoothing"""
        if self.n == 1:
            # For unigrams, use continuation counts
            if hasattr(self, 'continuation_counts'):
                total_continuations = sum(self.continuation_counts.values())
                return self.continuation_counts.get(ngram, 0) / total_continuations if total_continuations > 0 else 1/self.vocab_size
            else:
                return self._laplace_probability(ngram)
        else:
            context = ngram[:-1]
            context_count = self.context_counts[context]
            ngram_count = self.ngram_counts[ngram]

            if context_count == 0:
                return 1 / self.vocab_size

            # Discounted probability
            discounted_count = max(ngram_count - self.discount, 0)
            prob = discounted_count / context_count

            # Add back-off probability (simplified)
            lambda_weight = self.discount * len([ng for ng in self.ngram_counts if ng[:-1] == context]) / context_count
            backoff_prob = self.continuation_counts.get(ngram[-1:], 0) / sum(self.continuation_counts.values()) if hasattr(self, 'continuation_counts') else 1/self.vocab_size

            return prob + lambda_weight * backoff_prob

    def _good_turing_probability(self, ngram):
        """Simplified Good-Turing smoothing"""
        count = self.ngram_counts[ngram]

        # Count of counts
        count_counts = defaultdict(int)
        for c in self.ngram_counts.values():
            count_counts[c] += 1

        if count == 0:
            # Use count of singletons
            return count_counts[1] / sum(self.ngram_counts.values()) if count_counts[1] > 0 else 1/self.vocab_size
        else:
            # Adjusted count using Good-Turing
            adjusted_count = (count + 1) * count_counts[count + 1] / count_counts[count] if count_counts[count] > 0 else count
            if self.n == 1:
                total_count = sum(self.ngram_counts.values())
                return adjusted_count / total_count
            else:
                context = ngram[:-1]
                context_count = self.context_counts[context]
                return adjusted_count / context_count if context_count > 0 else 0

    def sentence_probability(self, sentence):
        """Calculate probability of a sentence"""
        if isinstance(sentence, str):
            sentence = sentence.lower().split()

        # Add sentence boundaries
        preprocessor = TextPreprocessor()
        padded_sentence = [preprocessor.start_token] * (self.n-1) + sentence + [preprocessor.end_token]

        log_prob = 0.0
        for i in range(len(padded_sentence) - self.n + 1):
            ngram = tuple(padded_sentence[i:i + self.n])
            prob = self.get_probability(ngram)
            if prob > 0:
                log_prob += log(prob)
            else:
                log_prob += log(1e-10)  # Small probability for zero probabilities

        return exp(log_prob)

    def perplexity(self, test_sentences):
        """Calculate perplexity on test sentences"""
        total_log_prob = 0.0
        total_words = 0

        for sentence in test_sentences:
            if isinstance(sentence, str):
                sentence = sentence.lower().split()

            # Handle unknown words
            processed_sentence = []
            for word in sentence:
                if word in self.vocabulary:
                    processed_sentence.append(word)
                else:
                    processed_sentence.append("<unk>")

            # Add sentence boundaries
            preprocessor = TextPreprocessor()
            padded_sentence = [preprocessor.start_token] * (self.n-1) + processed_sentence + [preprocessor.end_token]

            for i in range(len(padded_sentence) - self.n + 1):
                ngram = tuple(padded_sentence[i:i + self.n])
                prob = self.get_probability(ngram)
                prob = max(prob, 1e-10)  # Avoid log(0)
                total_log_prob += log(prob)
                total_words += 1

        avg_log_prob = total_log_prob / total_words if total_words > 0 else 0
        return exp(-avg_log_prob)

print("Training models with different configurations...")

# Initialize models
models = {
    'bigram_laplace': NGramModel(2, 'laplace', alpha=1.0),
    'bigram_kneser_ney': NGramModel(2, 'kneser_ney', discount=0.75),
    'trigram_laplace': NGramModel(3, 'laplace', alpha=1.0),
    'trigram_kneser_ney': NGramModel(3, 'kneser_ney', discount=0.75),
    'unigram_laplace': NGramModel(1, 'laplace', alpha=1.0)
}

# Train all models
for name, model in models.items():
    model.train(train_sentences)
    print(f"Trained {name}")

print("\nAll models trained successfully!")


# Test sentences for evaluation
test_sample_sentences = [
    "the cat sat on the mat",
    "natural language processing is fascinating",
    "machine learning algorithms are powerful",
    "the quick brown fox jumps",
    "artificial intelligence will change the world"
]

print("Evaluating sentence probabilities...")
results = []

for sentence in test_sample_sentences:
    sentence_results = {'sentence': sentence}
    for name, model in models.items():
        try:
            prob = model.sentence_probability(sentence)
            sentence_results[f'{name}_prob'] = prob
        except:
            sentence_results[f'{name}_prob'] = 0.0
    results.append(sentence_results)

# Create results DataFrame
results_df = pd.DataFrame(results)
print("\nSentence Probability Results:")
print(results_df.round(10))

# Calculate perplexity on test set
print("\nCalculating perplexity on test set...")
perplexity_results = {}

for name, model in models.items():
    try:
        perp = model.perplexity(test_sentences[:100])  # Use first 100 test sentences
        perplexity_results[name] = perp
        print(f"{name}: {perp:.2f}")
    except Exception as e:
        print(f"Error calculating perplexity for {name}: {e}")
        perplexity_results[name] = float('inf')


# Plot perplexity comparison
plt.figure(figsize=(12, 6))
model_names = list(perplexity_results.keys())
perplexities = [perplexity_results[name] for name in model_names]

# Filter out infinite values for plotting
finite_results = [(name, perp) for name, perp in zip(model_names, perplexities) if perp != float('inf')]
if finite_results:
    names, perps = zip(*finite_results)
    plt.bar(names, perps)
    plt.title('Model Perplexity Comparison')
    plt.xlabel('Model')
    plt.ylabel('Perplexity')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Analyze smoothing effects
print("\n" + "="*60)
print("SMOOTHING ANALYSIS")
print("="*60)

# Compare smoothing techniques
bigram_laplace = models['bigram_laplace']
bigram_kneser_ney = models['bigram_kneser_ney']

# Test with seen and unseen n-grams
test_ngrams = [
    ('the', 'cat'),
    ('cat', 'sat'),
    ('sat', 'on'),
    ('xyz', 'abc'),  # Unseen n-gram
    ('<s>', 'the'),
    ('the', '</s>')
]

print("Probability comparison for different n-grams:")
print(f"{'N-gram':<15} {'Laplace':<12} {'Kneser-Ney':<12}")
print("-" * 40)

for ngram in test_ngrams:
    laplace_prob = bigram_laplace.get_probability(ngram)
    kn_prob = bigram_kneser_ney.get_probability(ngram)
    print(f"{str(ngram):<15} {laplace_prob:<12.8f} {kn_prob:<12.8f}")


print("\n" + "="*60)
print("ERROR ANALYSIS")
print("="*60)

# Analyze why certain models perform better
print("1. Vocabulary Coverage Analysis:")
vocabulary_stats = {}
for name, model in models.items():
    vocabulary_stats[name] = {
        'vocab_size': model.vocab_size,
        'ngram_types': len(model.ngram_counts)
    }

vocab_df = pd.DataFrame(vocabulary_stats).T
print(vocab_df)

print("\n2. Sentence Length Impact on Perplexity:")
# Group test sentences by length and calculate perplexity
length_groups = defaultdict(list)
for sentence in test_sentences[:50]:
    length_groups[len(sentence)].append(sentence)

for length, sents in length_groups.items():
    if len(sents) >= 3:  # Only analyze groups with sufficient samples
        perp = models['bigram_laplace'].perplexity(sents)
        print(f"Length {length}: {perp:.2f} (n={len(sents)} sentences)")

print("\n3. Impact of Smoothing Parameters:")
# Test different alpha values for Laplace smoothing
alphas = [0.1, 0.5, 1.0, 2.0, 5.0]
alpha_perplexities = []

for alpha in alphas:
    model = NGramModel(2, 'laplace', alpha=alpha)
    model.train(train_sentences[:500])  # Use subset for faster training
    perp = model.perplexity(test_sentences[:50])
    alpha_perplexities.append(perp)
    print(f"Alpha = {alpha}: Perplexity = {perp:.2f}")

# Plot alpha vs perplexity
plt.figure(figsize=(10, 6))
plt.plot(alphas, alpha_perplexities, 'bo-')
plt.title('Impact of Laplace Smoothing Parameter on Perplexity')
plt.xlabel('Alpha Value')
plt.ylabel('Perplexity')
plt.grid(True)
